<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
</div>

* lire et écrire un fichier est très facile en Python
* ouvrir un fichier pour créer un objet "fichier"
* `open('mon_fichier.txt', 'r')`
  * `'r'` ouvre le fichier en lecture (défaut),
  * `‘w’` en écriture,
  * `‘a’` en écriture à la suite (*append*),

### utilisez un `with`

* prenez l'habitude de **toujours utiliser un context manager** 

In [ ]:
# on n'a pas encore étudié l'instruction with
# mais je vous conseille de toujours procéder comme ceci

# avec with on n'a pas besoin de fermer e fichier
with open('temporaire.txt', 'w') as f:
    for i in 10, 20, 30:
        f.write(f'{i} {i**2}\n')

In [ ]:
!cat temporaire.txt

# lecture

* l'objet fichier est un **itérable** lui-même
* on peut faire un `for` dessus

In [ ]:
# lire un fichier texte ligne par ligne
# on ne peut pas faire plus compact et lisible !

# remarquez aussi:
# open() sans le mode ⇔ open('r')

with open('temporaire.txt') as f:
    for line in f:
        print(f"-- {line}", end='') 

# fichiers texte ou binaire

* ajouter `'b'` au mode pour ouvrir **en binaire**
  * pas de décodage
  * travaille alors à base de **`bytes`** et non de `str`

In [ ]:
# j'ai besoin d'un objet bytes
# rappelez vous la section sur Unicode
text = "noël en été\n"
binaire = text.encode(encoding="utf-8")

binaire

In [ ]:
# remarquez le 'b' dans le mode d'ouverture

with open('temporaire.bin', 'wb') as out_file:
    # je peux du coup écrire un objet bytes
    out_file.write(binaire)

In [ ]:
!cat temporaire.bin

In [ ]:
# pareil en lecture, le mode avec un 'b'
# va faire que read() retourne un objet bytes

with open('temporaire.bin', 'rb') as in_file:
    binaire2 = in_file.read()

In [ ]:
# et donc on retombe bien sur nos pieds
binaire2 == binaire

# le module `pathlib`

## objectifs

* simplifier la gestion des noms de fichier 
* pour rendre le code plus concis
* et donc plus lisible
* sous-titre: *object-oriented filesystem paths*

## présentation du module

* voir [documentation complète](https://docs.python.org/3/library/pathlib.html)
* et notamment un diagramme des classes 
  * `purepath` : manipulation sans le filesystem
  * `path` : par exemple pour globbing (résoudre '*')
* dispo dans librairie standard depuis python-3.4
  * et aussi dans pypi, donc pour 2.7
* ne gère pas
  * les objets fichier (s'arrête à `open`)
  * les urls

## un exemple

* orienté objet
* le sujet devient plus visible
* **NB**: un objet `Path` est immutable

In [ ]:
# avec os.path
import os.path

config_dir = "/etc/apache2"
if os.path.isdir(config_dir):
    print("OUI")

In [ ]:
# avec pathlib
from pathlib import Path

config_path = Path("/etc/apache2")
if config_path.is_dir():
    print("OUI")

## l'opérateur `/`

* la fin de `os.path.join`

In [ ]:
# un chemin absolu
prefix = Path("/etc")
# le chemin absolu du directory courant
dot = Path.cwd()
# ou du homedir
home = Path.home()
# un nom de ficher
filename = Path("apache")

# Path / Path -> Path bien sûr
type(prefix / filename)

In [ ]:
# Path / str -> Path
type(prefix / "apache2")

In [ ]:
# str / Path -> Path
type("/etc" / Path("apache2"))

In [ ]:
# On peut chainer le tout sans parenthèse 
# si le premier (à gauche) est un Path

type(prefix / "apache2" / "modules.d")

In [ ]:
# mais bien sûr str / str -> TypeError
try:
    "/etc" / "apache2"
except Exception as e:
    print("OOPS", e)

## décorticage

* remplacement de `basename` et `dirname` et similaires

In [ ]:
# un chemin vers le directory 'filepath-globbing' dans ce répertoire

absolute = Path.cwd()
relative = Path(".")

globbing = absolute / "filepath-globbing"

# retrouver le string
str(globbing)

In [ ]:
globbing.parts

In [ ]:
# basename
globbing.name

In [ ]:
# dirname
globbing.parent

In [ ]:
list(globbing.parents)

In [ ]:
# parce qu'on l'a construit à partir de cwd() qui est absolu
globbing.is_absolute()

In [ ]:
Path("globbing").is_absolute()

In [ ]:
# ancien abspath()
globbing.resolve()

In [ ]:
list(globbing.parents)[-2]

In [ ]:
# ancien relpath()
# juste pour rendre le notebook utilisable partout (windows?)
level1 = list(globbing.parents)[-2]
print("level1", level1)
# chez moi level1 vaut "/Users"
globbing.relative_to(level1)

## pattern-matching

In [ ]:
# est-ce que le nom de mon objet Path 
# a une certaine forme ?

globbing.match("**/slides/*")

In [ ]:
globbing.match("**/*globbing*")

## pattern-matching

recherche dans un répertoire

In [ ]:
# un répertoire qui contient quelques fichiers
!ls filepath-globbing/**

In [ ]:
# à présent c'est plus intéressant
# avec des chemins relatifs
globbing = Path(".") / "filepath-globbing"

list(globbing.glob("*"))

In [ ]:
list(globbing.glob("*[0-9]"))

In [ ]:
list(globbing.glob("**"))

In [ ]:
list(globbing.glob("**/*[0-9]"))

In [ ]:
str(globbing)

## voir aussi

* `exists`, `is_dir`, `is_file` ...
* `stat` / `lstat` / `owner` pour les détails comme taille, permissions...
* `rename`, `unlink`, `rmdir` 
* `iterdir` (`os.listdir`, mais pas `os.walk`)
* `glob` - `rglob` 
* `open` / `{read,write}_{text_bytes}` / : wrappers 
* à nouveau: [documentation complète](https://docs.python.org/3/library/pathlib.html)

## remplacement pour

* le plus gros de `os.path`
* certaines choses de `os`
* `glob.glob`
* `fnmatch`
* contient un wrapper pour `open`